This notebook is made to check the presence of CyCOGs within species trees of groups of cyanobacteria. The other notebook makes gene trees, this notebook makes labels to check the gene within premade species trees.

In [1]:
from matplotlib import pyplot as plt
import os
import matplotlib as mpl
import numpy as np
import pandas as pd
import copy
import csv
from collections import Counter

from Bio import Phylo


In [5]:
# Pick CyCOG to generate labels for
CYCOG = 60000046

In [2]:
# Set dependencies
REFS = '../data/genomes/'
CYCOGS = 'data/0/serralysin_cycog_references.csv'
CYCOG_LIST = 'data/0/cycogs.tsv'
GENOMES = 'data/0/cycogsgenomes.tsv'
CLADE_MAP = 'data/0/updated-genome-clades.csv'
SEQ = "faa"

In [6]:
# make dataframe containing information about the CyCOG

# 1 & 2: extract protein IDs and associated genome names for all proteins in the CyCOG
cycog_df = pd.read_csv(CYCOG_LIST, sep="\t")
protein_ids = [] # empty list to store protein IDs
for i, row in cycog_df.iterrows(): # searches through rows in cycogs.tsv
    if CYCOG == int(row['cycog_iid'].split('_')[1]):
        print(row)
        protein_list = row['cycog_genes'].split(',')
label_df = pd.DataFrame(protein_list, columns=['protein_id']) # make a dataframe with a protein_id column
# check out rsplit documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.rsplit.html#pandas.Series.str.rsplit
label_df[['genome_name', 'gene_id']] = label_df['protein_id'].str.rsplit(pat='_', n=1, expand=True) # add new columns to the dataframe, the two components of
# protein_id separated using the underscore

# 3: join genome id on genome name
genomes_df = pd.read_csv(GENOMES, sep='\t') # reading in a new dataframe with genome data
label_df = pd.merge(label_df, genomes_df, left_on='genome_name', right_on='IID', how='left') # merging the two dataframes based on the genome_name in the
# label_df and the IID in the genomes_df, which should match

# 4: join clade on genome id
clade_df = pd.read_csv(CLADE_MAP) # read in new dataframe with genome IDs and clade information
label_df = pd.merge(label_df, clade_df, left_on='IMG_ID', right_on='IMGGenomeID', how='left') # merge dataframes matching IMG_ID with IMGGENOMEID

label_df

cycog_iid                                                    CyCOG_60000046
cycog_num_taxa                                                          497
cycog_num_genes                                                         946
cycog_num_duplications                                                  449
cycog_num_pro                                                           809
cycog_num_syn                                                           132
cycog_num_phage                                                           5
cycog_cns_product                               UDP-glucuronate 4-epimerase
cycog_genes               WH8016_2507493065,AG-442-B03_2717726087,scB245...
Name: 45, dtype: object


,protein_id,genome_name,gene_id,IID,GROUP,IMG_ID,TYPE,JGI_GENOMEPORTAL_NAME,Completeness,IMGGenomeID,UpdatedIMGGenomeID,Clade
0,WH8016_2507493065,WH8016,2507493065,WH8016,Synechococcus,2507262052,ISOLATE,Synechococcus_sp._WH_8016,99.18,2.507262e+09,2.507262e+09,5.1B-I
1,AG-442-B03_2717726087,AG-442-B03,2717726087,AG-442-B03,Prochlorococcus,2716884782,SAG,Uncultured_Prochlorococcus_sp._AG-442-B03,93.75,2.716885e+09,2.716885e+09,HLII
2,scB245a_521K15_2653032043,scB245a_521K15,2653032043,scB245a_521K15,Prochlorococcus,2651869831,SAG,Prochlorococcus_sp._scB245a_521K15,86.46,2.651870e+09,2.651870e+09,HLII
3,P-SSM2_638311613,P-SSM2,638311613,P-SSM2,Virus,638275926,ISOLATE,Prochlorococcus_phage_P-SSM2,0.00,NaN,NaN,NaN
4,AG-402-M23_2667747375,AG-402-M23,2667747375,AG-402-M23,Prochlorococcus,2667527274,SAG,Prochlorococcus_sp._AG-402-M23,84.38,2.667527e+09,2.667527e+09,LLI
...,...,...,...,...,...,...,...,...,...,...,...,...
941,AG-363-I21_2667895488,AG-363-I21,2667895488,AG-363-I21,Prochlorococcus,2667527369,SAG,Prochlorococcus_sp._AG-363-I21,68.49,2.667527e+09,2.667527e+09,LLII.LLIII
942,AG-418-G18_2717711922,AG-418-G18,2717711922,AG-418-G18,Prochlorococcus,2716884770,SAG,Uncultured_Prochlorococcus_sp._AG-418-G18,72.54,2.716885e+09,2.716885e+09,HLII
943,AG-347-M18_2667707121,AG-347-M18,2667707121,AG-347-M18,Prochlorococcus,2667527251,SAG,Prochlorococcus_sp._AG-347-M18,85.78,2.667527e+09,2.667527e+09,HLII
944,AG-418-L19_2717319812,AG-418-L19,2717319812,AG-418-L19,Prochlorococcus,2716884407,SAG,Uncultured_Prochlorococcus_sp._AG-418-L19,49.59,2.716884e+09,2.716884e+09,HLII


'60000046-check.tsv'

In [50]:
# write small files to use in eventual output file
with open("lgnd.txt", "w") as f:
    f.write(f'DATASET_LABEL\t{CYCOG}')
with open("title.txt", "w") as g:
    g.write(f'LEGEND_TITLE\t{CYCOG}')

In [51]:
# concatenate files referenced in this notebook, remove unnecessary files for directory cleaning
file_list = ['genelabel_temp1.txt', 'lgnd.txt', 'genelabel_temp3.txt', 'title.txt', 'genelabel_temp5.txt', cycog_check]
output_file = os.path.join(f'data/{CYCOG}/', f'{CYCOG}_labels.txt')
with open(output_file, 'w') as outfile:
    for fname in file_list:
        with open(fname, 'r') as infile:
            outfile.write(infile.read())
            outfile.write('\n')
os.remove('lgnd.txt')
os.remove('title.txt')
os.remove(cycog_check)

Code after this part is to generate clade labels for the species trees

In [19]:
GENOMES = 'data/0/cycogsgenomes.tsv'
CLADE_MAP = 'data/0/updated-genome-clades.csv'

In [11]:
genomes_df = pd.read_csv(GENOMES, sep='\t')
clade_df = pd.read_csv(CLADE_MAP)
label_df = pd.merge(genomes_df, clade_df, left_on='IMG_ID', right_on='IMGGenomeID', how='left') # merge dataframes matching IMG_ID with IMGGENOMEID

label_df

,IID,GROUP,IMG_ID,TYPE,JGI_GENOMEPORTAL_NAME,Completeness,IMGGenomeID,UpdatedIMGGenomeID,Clade
0,AG-311-D23,Prochlorococcus,2716884681,SAG,Uncultured_Prochlorococcus_sp._AG-311-D23,72.96,2.716885e+09,2.716885e+09,LLI
1,AG-311-I02,Prochlorococcus,2716884682,SAG,Uncultured_Prochlorococcus_sp._AG-311-I02,11.16,2.716885e+09,2.716885e+09,LLI
2,AG-311-I09,Prochlorococcus,2716884683,SAG,Uncultured_Prochlorococcus_sp._AG-311-I09,47.41,2.716885e+09,2.716885e+09,HLI
3,AG-311-J05,Prochlorococcus,2716884684,SAG,Uncultured_Prochlorococcus_sp._AG-311-J05,34.64,2.716885e+09,2.716885e+09,HLI
4,AG-311-J23,Prochlorococcus,2716884685,SAG,Uncultured_Prochlorococcus_sp._AG-311-J23,77.90,2.716885e+09,2.716885e+09,LLI
...,...,...,...,...,...,...,...,...,...
765,S-ShM2,Virus,651703106,ISOLATE,Synechococcus_phage_S-ShM2,0.00,NaN,NaN,NaN
766,Syn19,Virus,651703107,ISOLATE,Synechococcus_phage_Syn19,0.00,NaN,NaN,NaN
767,Syn5,Virus,641201056,ISOLATE,Synechococcus_phage_Syn5,0.00,NaN,NaN,NaN
768,metaG-MbCM1,Virus,2595698410,ISOLATE,Synechococcus_phage_metaG-MbCM1,0.00,NaN,NaN,NaN


In [27]:
label_df['Clade'] = label_df['Clade'].astype(str)

for clade in label_df:
    if label_df['Clade'].startswith('A'):
        print(label_df['IID'])
    elif label_df['Clade'].startswith('L'):
        print(clade)

AttributeError: 'Series' object has no attribute 'startswith'

# Prune genomic trees to only include branches in the Berube et al (2018) reference genomes

## Inputs: 
- Newick files of genome phylogenies (from Paul)
- Names of genomes in CyCOGv6
- Clades of genomes in CyCOGv6

## Outputs:
- Pruned newick files of genome phylogenies
- Label files for visualizing pruned phylogenies


In [8]:
# read in the genome phylogenies

hl_tree = Phylo.read('data/0/phylogeny/trees/HLprochlorococcus_5PERCto424cycogs_50percgaps_100boot_barnacle.nwk', 'newick')
ll_tree = Phylo.read('data/0/phylogeny/trees/LLprochlorococcus_5PERCto424cycogs_50percgaps_100boot_barnacle.nwk', 'newick')
syn_tree = Phylo.read('data/0/phylogeny/trees/synechococcus_5PERCto424cycogs_50percgaps_100boot_barnacle.nwk', 'newick')


In [ ]:
# identify leaves in tree that are in CyCOGv6

Phylo.draw_ascii(syn_tree)

In [12]:
# prune leaves from tree that aren't in CyCOGv6

tree_copy = copy.deepcopy(syn_tree)
for leaf in tree_copy.get_terminals():
    if leaf.name in label_df['IID'].tolist():
        continue
    else:
        tree_copy.prune(leaf.name)

Phylo.draw_ascii(tree_copy)

 , AG-676-A21
 |
 , AG-673-D02
 |
 | AG-676-E23
 |
 , MIT9508
 |
 |  _ AG-686-D09
 |,|
 ||| AG-679-A04
 ||
 || , MIT9504
 ||_|
 || | MIT9509
 ||
 ||              _ WH7803
 ||         ____|
 ,|        |    |___ WH7805
 ||        |
_||        |              , AG-679-C18
 ||        |           ___|
 ||        |          |   | AG-323-G20
 ||        |          |
 ||        |   _______|  _ WH8016
 ||        |  |       | |
 ||        |  |       | |, AG-686-A03
 ||        |  |       |_,|
 ||        |  |         ||, AG-683-A02
 ||        |  |         |||
 ||        |  |         | | AG-686-A05
 ||        |  |         |
 ||        |  |         | , AG-683-A03
 ||        |  |         |,|
 ||        |  |         ||| AG-686-F08
 ||        |  |         ||
 ||        |  |          , CC9311
 ||        |  |          |
 ||        |  |          |_ WH8020
 ||        |  |
 ||        |  |             ______ KORDI-49
 ||        |  |            |
 ||________|  |            |           , AG-676-F02
 |         | 

In [14]:
strains = label_df['IID'].tolist()

In [ ]:
Phylo.draw_ascii(syn_tree)

In [ ]:
test_copy = copy.deepcopy(syn_tree)

test_copy.prune('BIOS-U3-1')

Phylo.draw_ascii(test_copy)

Obsolete code below, delete when finished with phylo code

In [52]:
mapping_df = label_df[['gene_id', 'IMG_ID', 'genome_name', 'Clade']].rename(columns={'IMG_ID': 'genome_id', 'Clade': 'clade'})
mapping_df['leaf_id'] = mapping_df['gene_id']
mapping_df = mapping_df[['leaf_id', 'gene_id', 'genome_id', 'genome_name', 'clade']]

mapping_df

,leaf_id,gene_id,genome_id,genome_name,clade
0,2507493065,2507493065,2507262052,WH8016,5.1B-I
1,2717726087,2717726087,2716884782,AG-442-B03,HLII
2,2653032043,2653032043,2651869831,scB245a_521K15,HLII
3,638311613,638311613,638275926,P-SSM2,NaN
4,2667747375,2667747375,2667527274,AG-402-M23,LLI
...,...,...,...,...,...
941,2667895488,2667895488,2667527369,AG-363-I21,LLII.LLIII
942,2717711922,2717711922,2716884770,AG-418-G18,HLII
943,2667707121,2667707121,2667527251,AG-347-M18,HLII
944,2717319812,2717319812,2716884407,AG-418-L19,HLII


In [7]:
clade_lgnd = {
    "5.2": "#003D30", "5.3": "#00E5F8", "5.1A-CRD2": "#5A0A33", "5.1A-II": "#005745", "5.1A-III": "#810D49", "5.1A-IV": "#00735C",
    "5.1A-UC-A-EnvC": "#AB0D61", "5.1A-unclassified": "#009175", "5.1A-WPC1": "#D80D7B", "5.1B-CRD1": "#00AF8E", "5.1B-I": "#FF2E95", 
    "5.1B-IX": "#00CBA7", "5.1B-V": "#FF78AD", "5.1B-VI": "#00EBC1", "5.1B-VIII": "#FFACC6", "HLI": "#86FFDE", "HLII": "#FFD7E1",
    "HLII.HLVI": "#00306F", "HLIII": "#460B70", "HLIII.HLIV.HLV": "#00489E", "HLIV": "#6B069F", "HLVI": "#005FCC", "LLI": "#8E06CD",
    "LLI.LLVIII": "#0079FA", "LLI.LLIII": "#B40AFC", "LLIV": "#009FFA", "LLVII": "#ED0DFD", "LLVIII": "#00C2F9", "Blank": "#FF66FD",
    "": "#FF66FD", "LLII.LLIII": "#CDE494", "Unclassified": "#FF66FD", "AMZ-II": "#AA8F66"
}

In [9]:

clade_count = (Counter(label_df['Clade']))

In [10]:
clade_count

Counter({'HLII': 227,
         'HLI': 114,
         'LLI': 106,
         nan: 89,
         'LLVII': 58,
         'LLII.LLIII': 36,
         'LLIV': 26,
         '5.1B-CRD1': 18,
         'HLVI': 11,
         '5.1B-I': 10,
         '5.1A-IV': 9,
         '5.1A-II': 8,
         '5.1A-CRD2': 8,
         'AMZ-II': 7,
         'HLIII': 6,
         '5.2': 5,
         'HLIV': 5,
         '5.3': 4,
         '5.1A-UC-A-EnvC': 4,
         'HLII.HLVI': 3,
         '5.1A-III': 3,
         '5.1A-unclassified': 2,
         'LLVIII': 2,
         'LLI.LLVIII': 2,
         'Unclassified': 1,
         'HLIII.HLIV.HLV': 1,
         '5.1A-WPC1': 1,
         '5.1B-IX': 1,
         '5.1B-V': 1,
         '5.1B-VI': 1,
         '5.1B-VIII': 1})

In [21]:
filtered_clade_count = Counter({key: value for key, value in clade_count.items() if value >= 4})

print(filtered_clade_count)

clade_keys_list = list(filtered_clade_count.keys())

print(clade_keys_list)

Counter({'LLI': 416, 'HLII': 69, 'HLI': 5, '5.1B-CRD1': 4})
['LLI', 'HLII', 'HLI', '5.1B-CRD1']


In [ ]:
if clade.startswith(("L", "A")):
# if clade.startswith(("H")):
# if clade[0].isdigit():
    

In [17]:
output_matches = "cyano_clades.tsv"
template_file = "clade_template_fin.txt"

# Open files to write data
with open("ones.txt", "w") as f, open("clades.txt", "w") as g, open("labels.txt", "w") as h:
    f.write('LEGEND_SHAPES\t')
    for key, value in clade_lgnd.items():
        if key.startswith(("L", "A")):
            f.write('1\t')
            g.write(f'{key}\t')
            h.write(f'{value}\t')

# Append additional legend information to ones.txt
with open("ones.txt", "a") as f, open("clades.txt", "r") as g, open("labels.txt", "r") as h:
    f.write(f'\nLEGEND_LABELS\t{g.read()}')
    f.write(f'\nLEGEND_COLORS\t{h.read()}')

# Clean up temporary files
os.remove('clades.txt')
os.remove('labels.txt')

# Read contents of the template files
with open('clade-template1.txt', 'r') as clade1:
    content1 = clade1.read()
with open('ones.txt', 'r') as clade2:
    content2 = clade2.read()
with open('clade-template3.txt', 'r') as clade3:
    content3 = clade3.read()

# Concatenate the contents with newline characters
cladestogether = content1 + "\n" + content2 + '\n' + content3

# Write the combined content to a new file
with open('clade_template_fin.txt', 'w') as template_clade:
    template_clade.write(cladestogether)

In [13]:
with open(output_matches, 'w', newline='') as outfile:  # Use 'w' mode to create a new file
    writer = csv.DictWriter(outfile, fieldnames=['ID', 'LABEL'], delimiter='\t')
    writer.writeheader()  # Write the header once at the beginning

    for index, row in label_df.iterrows():
        leaf_id = row['IID']
        clade = row['Clade']
        clade = str(clade)
        if clade.startswith(("L", "A")):
            value = clade_lgnd.get(clade)  # Use default color if clade not found
        
            writer.writerow({'ID': leaf_id, 'LABEL': value})

print(output_matches, "generated successfully.")

60000046-clade.tsv generated successfully.


In [18]:
file_list = ['clade-template1.txt', 'ones.txt', 'clade-template3.txt', 'cyano_clades.tsv']
output_file = os.path.join('data/0/', 'LL_clades.txt')
with open(output_file, 'w') as outfile:
    for fname in file_list:
        with open(fname, 'r') as infile:
            outfile.write(infile.read())
            outfile.write('\n')
os.remove('ones.txt')
os.remove(output_matches)
# os.remove(cycog_check)

In [16]:
# Join previously made tsv with template file to complete annotation file for iTOL
# Read the content of the output file
# output_template_file = os.path.join(f'data/{CYCOG}/', f"{CYCOG}-gene_clades.txt")
output_template_file = os.path.join(f'data/0/', "LL_clades.txt")

with open(output_matches, 'r') as output_f:
    output_content = output_f.read()

# Read the content of the template file and append the output content
with open(template_file, 'r') as template_f:
    template_content = template_f.read()
    # Add a newline character between the template and output content
    combined_content = template_content + "\n" + output_content

# Write the combined content to a new file
with open(output_template_file, 'w') as combined_f:
    combined_f.write(combined_content)


os.remove('ones.txt')
# os.remove('clade_template_fin.txt')

print(CYCOG, "annotation file generated successfully:", output_template_file, '\n')
os.remove(output_matches)
print ("Old outputs removed to clean directory. To run this block again generate a new output_matches file.") 
# output_matches file used to produce this block's output is removed for directory cleaning.

FileNotFoundError: [Errno 2] No such file or directory: 'clade_template_fin.txt'